# txt to xml

In [1]:
from tqdm import tqdm
from PIL import Image
from glob import glob
import numpy as np
import shutil
from sklearn.model_selection import train_test_split

In [2]:
path = glob('/media/localley/D/data/images/*.jpg')

In [4]:
for i in tqdm(path):
    image = Image.open(i)
    name = i.split('/')[-1]
    img = np.array(image)
    h, w = img.shape[0], img.shape[1]
    with open(i.replace('jpg','txt'), 'r') as f:
        lines = f.readlines()
        g = open(i.replace('jpg','xml'), 'w')
        g.write('<annotation>\n')
        g.write('	<folder>images</folder>\n')
        g.write('	<filename>{}</filename>\n'.format(name))
        g.write('	<path>{}</path>\n'.format(i))
        g.write('	<source>\n')
        g.write('		<database>Unknown</database>\n')
        g.write('	</source>\n')
        g.write('	<size>\n')
        g.write('		<width>{}</width>\n'.format(w))
        g.write('		<height>{}</height>\n'.format(h))
        g.write('		<depth>3</depth>\n')
        g.write('	</size>\n')
        g.write('	<segmented>0</segmented>\n')
        for line in lines:
            li = line.split()
            if li[0] == '0':
                cate = 'dog'
            elif li[0] == '1':
                cate = 'person'
            else:
                cate = 'cat'
            xmin = int((float(li[1]) - float(li[3])/2) * w)
            ymin = int((float(li[2]) - float(li[4])/2) * h)
            xmax = int((float(li[1]) + float(li[3])/2) * w)
            ymax = int((float(li[2]) + float(li[4])/2) * h)
            if float(li[1]) * float(li[2]) < 1/(16*16):
                continue
            g.write('	<object>\n')
            g.write('		<name>{}</name>\n'.format(cate))
            g.write('		<pose>Unspecified</pose>\n')
            g.write('		<truncated>0</truncated>\n')
            g.write('		<difficult>0</difficult>\n')
            g.write('		<bndbox>\n')
            g.write('			<xmin>{}</xmin>\n'.format(xmin))
            g.write('			<ymin>{}</ymin>\n'.format(ymin))
            g.write('			<xmax>{}</xmax>\n'.format(xmax))
            g.write('			<ymax>{}</ymax>\n'.format(ymax))
            g.write('		</bndbox>\n')
            g.write('	</object>\n')
        g.write('</annotation>')
    g.close()
    f.close()

100%|██████████| 33017/33017 [10:08<00:00, 54.26it/s]


# txt to yolov5 dataset

In [5]:
train, test = train_test_split(path, random_state = 123)

In [6]:
for i in tqdm(train):
    shutil.copy(i,'/media/localley/D/data/v5image/image/train')
    shutil.copy(i.replace('jpg','txt'),'/media/localley/D/data/v5image/image/train')

100%|██████████| 24762/24762 [00:57<00:00, 433.60it/s] 


In [7]:
for i in tqdm(test):
    shutil.copy(i,'/media/localley/D/data/v5image/image/test')
    shutil.copy(i.replace('jpg','txt'),'/media/localley/D/data/v5image/image/test')

100%|██████████| 8255/8255 [00:58<00:00, 141.10it/s]


# voc data format

In [6]:
path = glob('/media/localley/D/data/voc_pet/images/*.jpg')

In [7]:
train, test = train_test_split(path, train_size = 0.8, random_state = 123)

In [8]:
train, val = train_test_split(train, test_size = 0.125, random_state = 123)

In [9]:
len(train), len(test), len(val)

(23111, 6604, 3302)

In [13]:
with open('/media/localley/D/data/voc_pet/train.txt','w') as f:
    for name in tqdm(train):
        f.write(name.split('/')[-1].replace('.jpg',''))
        f.write('\n')
f.close()

100%|██████████| 23111/23111 [00:00<00:00, 853567.68it/s]


In [14]:
with open('/media/localley/D/data/voc_pet/test.txt','w') as f:
    for name in tqdm(test):
        f.write(name.split('/')[-1].replace('.jpg',''))
        f.write('\n')
f.close()

100%|██████████| 6604/6604 [00:00<00:00, 964422.67it/s]


In [15]:
with open('/media/localley/D/data/voc_pet/val.txt','w') as f:
    for name in tqdm(val):
        f.write(name.split('/')[-1].replace('.jpg',''))
        f.write('\n')
f.close()

100%|██████████| 3302/3302 [00:00<00:00, 892543.13it/s]
